In [1]:
from gensim.models.phrases import Phrases
import pandas as pd

In [2]:
phrase_dfs = {
    f"{n}gram_df" : pd.DataFrame(
        Phrases.load(f"gensim_phrase_models/{n}gram_phrases.pkl").export_phrases().items(),
        columns =["Phrase", "Score"]
    ).sort_values(by="Score")
    for n in ["bi", "tri", "four"]
}

bigram_df = phrase_dfs["bigram_df"]
trigram_df = phrase_dfs["trigram_df"]
fourgram_df = phrase_dfs["fourgram_df"]

del phrase_dfs

In [8]:
bigram_df[bigram_df["Phrase"] == "reserve_bank"]

,Phrase,Score
15,reserve_bank,0.56248
